In [20]:
import openai

In [21]:
import minsearch

In [22]:
import json

In [23]:
with open("documents.json", "rt") as f_in:
    docs_raw = json.load(f_in)

In [24]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [25]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [26]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [27]:
index

In [28]:
q = 'the course has already started, can I still enroll?'

In [29]:
index.fit(documents)

In [30]:
boost = {'question' : 3.0, 'section' : 0.5}
results = index.search(
    query = q,
    filter_dict = {'course' : 'data-engineering-zoomcamp'},
    boost_dict = boost,
    num_results = 5
)

In [31]:
from openai import OpenAI

In [32]:
client = OpenAI()

In [33]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{'role': 'user', "content": q}]
)

In [34]:
response.choices[0].message.content

"Whether you can still enroll in a course after it has started depends on the specific policies of the institution offering the course. Here are a few steps you can take to find out if it's possible:\n\n1. **Check the Institution’s Policy**: Look on the course or institution’s website for information on their enrollment policy. Some institutions allow late registration within a certain time frame.\n\n2. **Contact the Instructor**: Reach out to the course instructor directly. They might allow late enrollment if you can catch up on missed work.\n\n3. **Speak with the Admissions Office**: The admissions or registrar’s office can provide guidance on late enrollment procedures and deadlines.\n\n4. **Online Courses**: If it's an online course, there might be more flexibility. Check if the course platform allows late joiners or provides recordings of past lectures.\n\n5. **Catch-Up Plan**: Be prepared to discuss how you plan to catch up on any missed material or assignments.\n\nAct quickly, a

In [37]:
context = ""

for doc in results:
    context = context + f"section: {doc['section']} \nquestion: {doc['question']} \ntext: {doc['text']}"

In [40]:
print(context)

section: General course-related questions 
question: Course - Can I still join the course after the start date? 
text: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.section: General course-related questions 
question: Course - Can I follow the course after it finishes? 
text: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.section: General course-related questions 
question: Course - When will the course start? 
text: The purpose of this document is to capture frequently asked technical questions
The exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with th

In [41]:
prompt_template = """ 
You are a course teaching assistant. Answer the QUESTION based on CONTEXT. 
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesnt contain the answer, output None.

QUESTION: {question}

CONTEXT: {context}
""".strip()

In [44]:
prompt = prompt_template.format(question=q, context=context).strip()

In [45]:
print(prompt)

You are a course teaching assistant. Answer the QUESTION based on CONTEXT. 
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesnt contain the answer, output None.

QUESTION: the course has already started, can I still enroll?

CONTEXT: section: General course-related questions 
question: Course - Can I still join the course after the start date? 
text: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.section: General course-related questions 
question: Course - Can I follow the course after it finishes? 
text: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.sectio

In [46]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{'role': 'user', "content": prompt}]
)

response.choices[0].message.content

'Yes, even if the course has already started, you can still enroll and are eligible to submit the homeworks. However, be aware of the deadlines for turning in the final projects.'